<a href="https://colab.research.google.com/github/guilhermelaviola/ApplicationsOfDataScienceInDisruptiveTechnologies/blob/main/Class02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Correlation-based algorithms**
Uses statistical correlation to make decisions, such as in feature selection, where it identifies features most related to the target variable while minimizing redundancy between features. Other applications include pattern recognition in image processing, using correlation to find matching templates, and recommendation systems, using correlation to measure user or item similarity. These algorithms measure the relationship between two or more variables to identify similarities or differences.

In [1]:
# Importing all the necessary libraries:
import cv2
import matplotlib.pyplot as plt
import numpy as np
from google.colab.patches import cv2_imshow  # to use cv2_imshow on Colab

In [2]:
# Function to display images in Colab:
def display_image(frame):
    img_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    plt.imshow(img_rgb)
    plt.axis('off')
    plt.show()

# Getting the name of the loaded video:
video_path = list(uploaded.keys())[0]
cap = cv2.VideoCapture(video_path)

# Reading the first video frame:
ret, frame = cap.read()

NameError: name 'uploaded' is not defined

## **Regression-based algorithms**
Subset of machine learning algorithms that predict a continuous output variable based on one or more input features. Regression aims to model the relationship between the dependent variable (output) and one or more independent variables (inputs).

In [ ]:
# Function to display images in Colab:
def display_image(frame):
    img_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    plt.imshow(img_rgb)
    plt.axis('off')
    plt.show()

# Loading video with OpenCV:
cap = cv2.VideoCapture('https://www.youtube.com/shorts/SxZJ4WWr6uE')  # Replace it with the video path

In [ ]:
# Reading the first frame of the video:
ret, frame = cap.read()
if not ret:
    print('Error loading the video')
else:
    # Defining a Region of Interest (ROI) manually:
    h, w, _ = frame.shape
    roi = (int(w/4), int(h/4), int(w/2), int(h/2))  # ROI maior para rastreamento

    # Displaying the first frame with the ROI:
    p1 = (int(roi[0]), int(roi[1]))
    p2 = (int(roi[0] + roi[2]), int(roi[1] + roi[3]))
    cv2.rectangle(frame, p1, p2, (255, 0, 0), 2, 1)
    exibir_imagem(frame)

    # Initializing the MOSSE tracker:
    tracker = cv2.legacy.TrackerMOSSE_create()

    # Initializing the tracker with the first frame and ROI:
    tracker.init(frame, roi)

    # Tracking the object in the video:
    while True:
        ret, frame = cap.read()
        if not ret:
            break

        # Updating the tracker
        ok, bbox = tracker.update(frame)

        if ok:
            # Drawing bounding box if tracking is successful:
            p1 = (int(bbox[0]), int(bbox[1]))
            p2 = (int(bbox[0] + bbox[2]), int(bbox[1] + bbox[3]))
            cv2.rectangle(frame, p1, p2, (255, 0, 0), 2, 1)
        else:
            # Displaying tracking failure message:
            cv2.putText(frame, 'Tracking failed', (100, 80), cv2.FONT_HERSHEY_SIMPLEX, 0.75, (0, 0, 255), 2)

        # Displaying the updated frame:
        display_image(frame)

    cap.release()

## **Optical Flow-Based Algorithms**
Optical Flow is a way to analyze a scene and provide movement information, in the form of speed vectors (i.e. direction and amplitude). It is well known for frame-based cameras, but given this new event-based paradigm, we adopt new approaches to achieve this goal, while preserving the asynchronous nature of events.

In [ ]:
# Loading the video:
cap = cv2.VideoCapture('video.avi')

# Defining parameters for the Lucas-Kanade algorithm:
lk_params = dict(winSize=(15, 15),
                 maxLevel=2,
                 criteria=(cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))

# Generating some random colors to draw the trails:
color = np.random.randint(0, 255, (100, 3))

# Reading the first frame and detecting the points of interest:
ret, old_frame = cap.read()
old_gray = cv2.cvtColor(old_frame, cv2.COLOR_BGR2GRAY)
p0 = cv2.goodFeaturesToTrack(old_gray, mask=None, maxCorners=100, qualityLevel=0.3, minDistance=7, blockSize=7)

In [ ]:
# Creating an image mask to draw the trails:
mask = np.zeros_like(old_frame)

while True:
    ret, frame = cap.read()
    if not ret:
        break
    frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Calculating the optical flow:
    p1, st, err = cv2.calcOpticalFlowPyrLK(old_gray, frame_gray, p0, None, **lk_params)

    # Selecting the good points:
    good_new = p1[st == 1]
    good_old = p0[st == 1]

    # Drawing the trails:
    for i, (new, old) in enumerate(zip(good_new, good_old)):
        a, b = new.ravel()
        c, d = old.ravel()
        mask = cv2.line(mask, (int(a), int(b)), (int(c), int(d)), color[i].tolist(), 2)
        frame = cv2.circle(frame, (int(a), int(b)), 5, color[i].tolist(), -1)

    img = cv2.add(frame, mask)

    # Displaying the frame with the tracks:
    cv2_imshow(img)

    # Updating the parameters for the next frame:
    old_gray = frame_gray.copy()
    p0 = good_new.reshape(-1, 1, 2)

    if cv2.waitKey(30) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()